<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml1027/blob/main/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model = InceptionV3(include_top=False, input_shape=(224, 224, 3))
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file="inception.png", show_shapes=True)

In [10]:
# 右一線: 1 x 1卷積(64)
# 64組 1 x 1捲機
w = model.get_layer("conv2d_5").get_weights()
print(w[0].shape)

(1, 1, 192, 64)


In [11]:
# 右二線: 模糊化(Average) -> 1 x 1卷積(32)
w = model.get_layer("conv2d_11").get_weights()
print(w[0].shape)

(1, 1, 192, 32)


In [12]:
# 右三線: 1 x 1捲機(48) -> 5 x 5捲機(64)
# 1 x 1 捲機: 減低計算量
w = model.get_layer("conv2d_6").get_weights()
print(w[0].shape)
w = model.get_layer("conv2d_7").get_weights()
print(w[0].shape)

(1, 1, 192, 48)
(5, 5, 48, 64)


In [13]:
# 右四線: 1 x 1捲機(64) -> 3 x 3捲機(96) -> 3 x 3捲機(96)
w = model.get_layer("conv2d_8").get_weights()
print(w[0].shape)
w = model.get_layer("conv2d_9").get_weights()
print(w[0].shape)
w = model.get_layer("conv2d_10").get_weights()
print(w[0].shape)

(1, 1, 192, 64)
(3, 3, 64, 96)
(3, 3, 96, 96)
